## CNN Feature Map Visualization

### Notebook & Packages

In [1]:
import helper as hp 
hp.initialize_notebook() # initialize with GPU enabled  
# hp.initialize_notebook(False) # to disable GPU 

2024-12-17 11:59:22.343450: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-17 11:59:22.358019: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-17 11:59:22.362542: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-17 11:59:22.373658: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


GPU enabled. Checking for available GPUs...
1 Physical GPUs, 1 Logical GPUs

Verifying TensorFlow and PyTorch CUDA setup...
TensorFlow version: 2.17.0
Built with CUDA: True
Num GPUs Available: 1

Keras version: 3.6.0

End checks and initialization.


I0000 00:00:1734436764.319135  716392 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1734436764.360633  716392 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1734436764.364167  716392 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1734436764.369296  716392 cuda_executor.cc:1015] successful NUMA node read from SysFS ha

In [3]:
# Import standard libraries
import os
import sys
import gc
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time
import keras
# Import DL libraries
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, Dropout # General Layers
from keras.layers import RandomRotation, RandomFlip
from keras import layers, models, Model, Input
from keras.utils import to_categorical
from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau, EarlyStopping, TensorBoard
from keras.initializers import glorot_uniform
from keras.utils import plot_model
# Suppress tensorflow warnings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# Import ML libraries
from sklearn.metrics import f1_score, confusion_matrix, ConfusionMatrixDisplay
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, StratifiedKFold, StratifiedShuffleSplit
from sklearn.utils.class_weight import compute_class_weight

# Import image libraries
from skimage import transform
import tifffile as tiff
from tifffile import imread

# Appends current working dir
current_path = os.getcwd()
sys.path.append(current_path)

# Import custom preprocessing class
from imc_preprocessing import IMCPreprocessor

#### Import Data

In [4]:
images_dir = '/home/jupyter-luvogt/Final_Project_LR/IMC_images' 
metadata_dir = '/home/jupyter-luvogt/Final_Project_LR/metadata.csv'
# --- Load CSV and filter dataset ---
csv_file = metadata_dir
image_folder = images_dir

# Load the CSV
df = pd.read_csv(csv_file)

# Filter rows with NA in PDL1_score and convert to binary
df = df.dropna(subset=["PDL1_score"])
df["PDL1_score"] = df["PDL1_score"].astype(int)

# --- Train-Test-Validation Split ---
train_df, val_and_test_df = train_test_split(df, test_size=0.4, random_state=42, stratify=df["PDL1_score"])
test_df, val_df = train_test_split(val_and_test_df, test_size=0.5, random_state=42, stratify=val_and_test_df["PDL1_score"])

# --- Load Images and Compute Mean and Standard Deviation ---

# Function to load a single image
def load_image(image_path):
    image = imread(image_path)  # Load all 46 channels
    return image.astype(np.float32)

# Initialize variables to accumulate sum and sum of squares
nr_images = 0
sum_images = np.zeros((46, 224, 224))
sum_squared_images = np.zeros((46, 224, 224))

# Accumulate the sum and sum of squares for the training dataset
for _, row in train_df.iterrows():
    image_path = os.path.join(image_folder, f"{row['sample_id']}.tiff")
    image = load_image(image_path)
    nr_images += 1
    sum_images += image
    sum_squared_images += image ** 2

# Compute mean and standard deviation
mean = sum_images / nr_images
std = np.sqrt(sum_squared_images / nr_images - mean ** 2)

# --- Dataset Loading Function ---

def load_dataset(dataframe, image_folder, normalize = True): # augment = False
    images = []
    labels = []
    
    for _, row in dataframe.iterrows():
        image_path = os.path.join(image_folder, f"{row['sample_id']}.tiff")
        image = load_image(image_path)

        if normalize:
            image = (image - mean) / std

        # Convert to channels-last format (224, 224, 46)
        image = np.transpose(image, (1, 2, 0))

        # # Apply data augmentation if specified
        # if augment:
        #     # Random horizontal flip
        #     if np.random.rand() > 0.5:
        #         image = np.flip(image, axis=2)
        #     # Random vertical flip
        #     if np.random.rand() > 0.5:
        #         image = np.flip(image, axis=1)

        images.append(image)
        labels.append(row["PDL1_score"])

    return np.array(images), np.array(labels)

### Create Datasets

In [5]:
# Training dataset without augmentation
X_train, y_train = load_dataset(train_df, image_folder, normalize = True)
# # Training dataset with data augmentation
# X_train_aug, y_train_aug = load_dataset(train_df, image_folder, augment=True)
# Validation dataset
X_val, y_val = load_dataset(val_df, image_folder, normalize = True)
# Test dataset
X_test, y_test = load_dataset(test_df, image_folder, normalize = True)

# One Hot Encoding
y_train= to_categorical(y_train)
y_val= to_categorical(y_val)
y_test = to_categorical(y_test)

print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
print(f"X_val shape: {X_val.shape}, y_val shape: {y_val.shape}")
print(f"X_test shape: {X_test.shape}, y_test shape: {y_test.shape}")

X_train shape: (590, 224, 224, 46), y_train shape: (590, 2)
X_val shape: (197, 224, 224, 46), y_val shape: (197, 2)
X_test shape: (197, 224, 224, 46), y_test shape: (197, 2)


### Extract last layer before classification

In [6]:
# Load model 
# To load the model from file uncomment and run the following 
model_ResNet18 = keras.saving.load_model('./models_ResNet/model_ResNet18_46_dropout_callbacks_trained.keras')

### Get Layer Names

In [7]:
layer_names = [layer.name for layer in model_ResNet18.layers]

We want dense2 layer to be extracted

### Get Layer Output

In [9]:
layer_outputs = [layer.output for layer in model_ResNet18.layers]

In [10]:
second_last_layer_output = layer_outputs[:-2] # Get all the layers except the last one (disgard the classification layer)

### Feature Map Model 

In [11]:
feature_map_model = Model(inputs=model_ResNet18.input, outputs=second_last_layer_output)

In [12]:
t = feature_map_model.predict(X_test)

I0000 00:00:1734436994.086797  716633 service.cc:146] XLA service 0x7fbc5d01c310 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1734436994.086854  716633 service.cc:154]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
2024-12-17 12:03:14.116270: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-12-17 12:03:14.258719: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 90300


3/7 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step

I0000 00:00:1734436997.756414  716633 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


5/7 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step

2024-12-17 12:03:28.165005: W external/local_tsl/tsl/framework/bfc_allocator.cc:482] Allocator (GPU_0_bfc) ran out of memory trying to allocate 341.62MiB (rounded to 358220800)requested by op 
2024-12-17 12:03:28.165086: I external/local_tsl/tsl/framework/bfc_allocator.cc:1039] BFCAllocator dump for GPU_0_bfc
2024-12-17 12:03:28.165096: I external/local_tsl/tsl/framework/bfc_allocator.cc:1046] Bin (256): 	Total Chunks: 107, Chunks in use: 107. 26.8KiB allocated for chunks. 26.8KiB in use in bin. 14.1KiB client-requested in use in bin.
2024-12-17 12:03:28.165102: I external/local_tsl/tsl/framework/bfc_allocator.cc:1046] Bin (512): 	Total Chunks: 48, Chunks in use: 48. 24.2KiB allocated for chunks. 24.2KiB in use in bin. 24.1KiB client-requested in use in bin.
2024-12-17 12:03:28.165107: I external/local_tsl/tsl/framework/bfc_allocator.cc:1046] Bin (1024): 	Total Chunks: 48, Chunks in use: 48. 48.8KiB allocated for chunks. 48.8KiB in use in bin. 48.0KiB client-requested in use in bin.
20

ResourceExhaustedError: Graph execution error:

Detected at node StatefulPartitionedCall defined at (most recent call last):
  File "/opt/tljh/user/envs/BME342_GPU_env/lib/python3.12/runpy.py", line 198, in _run_module_as_main

  File "/opt/tljh/user/envs/BME342_GPU_env/lib/python3.12/runpy.py", line 88, in _run_code

  File "/opt/tljh/user/envs/BME342_GPU_env/lib/python3.12/site-packages/ipykernel_launcher.py", line 18, in <module>

  File "/opt/tljh/user/envs/BME342_GPU_env/lib/python3.12/site-packages/traitlets/config/application.py", line 1075, in launch_instance

  File "/opt/tljh/user/envs/BME342_GPU_env/lib/python3.12/site-packages/ipykernel/kernelapp.py", line 739, in start

  File "/opt/tljh/user/envs/BME342_GPU_env/lib/python3.12/site-packages/tornado/platform/asyncio.py", line 205, in start

  File "/opt/tljh/user/envs/BME342_GPU_env/lib/python3.12/asyncio/base_events.py", line 641, in run_forever

  File "/opt/tljh/user/envs/BME342_GPU_env/lib/python3.12/asyncio/base_events

In [13]:
X_test[0].shape

(224, 224, 46)

### t-SNE on input data with 46 channels